<a href="https://colab.research.google.com/github/vnavya2004/Depressiondetection_BTP/blob/main/english_depressiondetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

uploaded = files.upload()
df = pd.read_csv("Binary_labeled_tweets.csv")
df.head(5)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], stratify=df['label'])

X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

# def get_sentence_embedding(sentences):
#     preprocessed_text = bert_preprocess(sentences)
#     return bert_encoder(preprocessed_text)['pooled_output']

# e = get_sentence_embedding([
#     "Sad",
#     "grapes",
#     "mango",
#     "jeff bezos",
#     "elon musk",
#     "bill gates"
# ])

# # Calculate cosine similarities
# print(cosine_similarity([e[0]], [e[1]]))
# print(cosine_similarity([e[0]], [e[3]]))
# print(cosine_similarity([e[3]], [e[4]]))

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

model.evaluate(X_test, y_test)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.7 MB/s eta 0:00:00


Saving Binary_labeled_tweets.csv to Binary_labeled_tweets.csv
[[0.9909136]]
[[0.88851726]]
[[0.9872034]]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                     

[0.41806045174598694, 0.8126703500747681]